In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from neuraldistributions.models import transforms
from neuraldistributions.datasets import static

from multiprocessing import Pool

import os
device = "cuda"

dataport not available, will only be able to load data locally


In [2]:
import numpy as np
import torch

from neuraldistributions.datasets import mouse_static_loaders, extract_data_key
from neuraldistributions.models import poisson, zig, flowfa, ziffa
from neuraldistributions.trainers import base_trainer
from neuraldistributions.utility import get_loglikelihood

In [3]:
random_seed=42
dataset_dir = "../project/data"
datasets = [
    "static_edited_dsampled.zip",
]
scan_id = [2, 1]
dataset_paths = [f"{dataset_dir}/{dataset}" for dataset in datasets]
areas = [["V1", "LM"]]
neurons_ns = [1000, 907]

In [4]:
dataset_index = 0
dataset_path = dataset_paths[dataset_index]
data_key = extract_data_key(dataset_path)
area = areas[dataset_index]
neurons_n = neurons_ns[dataset_index]

dataset_config = {
    "paths": [dataset_path],
    "seed": random_seed,
    "batch_size": 64,
    "area": area,
    "neurons_n": neurons_n,
    "normalize_images": True,
    "normalize_neurons": True,
    "return_more": True,
    "device": device,
    "shuffle_train": True,
}

dataloaders = mouse_static_loaders(**dataset_config)

100%|██████████| 5994/5994 [00:00<00:00, 288959.80it/s]


In [5]:
grid_mean_pred_config = {
    "type": "cortex",
    "input_dimensions": 2,
    "hidden_layers": 4,
    "hidden_features": 20,
    "nonlinearity": "ReLU",
    "final_tanh": True,
}

image_model_config = {
    "pad_input": False,
    "stack": -1,
    "layers": 4,
    "input_kern": 15,
    "gamma_input": 1,
    "gamma_readout": 2.439,
    "hidden_dilation": 1,
    "hidden_kern": 13,
    "hidden_channels": 64,
    "n_se_blocks": 0,
    "depth_separable": True,
    "share_features": False,
    "share_grid": False,
    "init_sigma": 0.4,
    "init_mu_range": 0.55,
    "gauss_type": "full",
    "grid_mean_predictor": grid_mean_pred_config,
}

In [6]:
model_name = "flowfa"
d_latent = 3
sample_transform = "learned-leaky-low-rank-k-500"
mean_transform = "learned"
per_neuron_transform = True
unit_variance_constraint = False

model_config = {
    "image_model_fn": "nnsysident.models.models.se2d_fullgaussian2d",
    "image_model_config": image_model_config,
    "d_latent": d_latent,
    "sample_transform": sample_transform,
    "mean_transform": mean_transform,
    "per_neuron_samples_transform": per_neuron_transform,
    "init_psi_diag_coef": 0.1,
    "init_C_coef": 0.1,
    "unit_variance_constraint": unit_variance_constraint,
}
model = flowfa(dataloaders, random_seed, **model_config)

In [7]:
index_name_of_dataloader = next(iter(dataloaders["train"]))
indexes = np.arange(len(dataloaders["image_ids"][index_name_of_dataloader]["test"]))
u, inv = (np.unique(dataloaders["image_ids"][index_name_of_dataloader]["test"], return_inverse=True))
img_indexes = indexes[inv == inv[0]]
print(img_indexes)

[0 1 2 3 4 5 6 7 8 9]


In [8]:
print(model.sample_transform)

Flow(
  (layers): ModuleList(
    (0): LowRankAffine()
    (1): ELU()
    (2): LowRankAffine()
    (3): ELU()
    (4): LowRankAffine()
    (5): ELU()
    (6): LowRankAffine()
  )
)


In [ ]:
trainer_config = {
    "epochs": 500,
    "device": "cuda",
    "lr": 0.005,
    "use_avg_loss": True
}
out = None
## The following line would train the model (upon availability of the dataset)
out = base_trainer(model, dataloaders, random_seed, **trainer_config)

Score improved (nan --> 5712.823242).


100%|██████████| 70/70 [00:05<00:00, 12.54it/s]


Epoch 1/500 | train loss: 4334.402832 | val loss: 5585.683594
Score improved (5712.823242 --> 5585.683594).


100%|██████████| 70/70 [00:05<00:00, 12.58it/s]


Epoch 2/500 | train loss: 4913.527832 | val loss: 5572.370605
Score improved (5585.683594 --> 5572.370605).


...

100%|██████████| 70/70 [00:05<00:00, 12.32it/s]


Epoch 132/500 | train loss: 1390.940063 | val loss: 1504.981323
EarlyStopping counter: 8 out of 50


100%|██████████| 70/70 [00:05<00:00, 12.31it/s]


Epoch 133/500 | train loss: 1384.411743 | val loss: 1509.582764
EarlyStopping counter: 9 out of 50


 51%|█████▏    | 36/70 [00:02<00:02, 12.27it/s]

In [ ]:
if out is not None:
    fig, ax = plt.subplots(1,5,figsize=(50,10))
    ax[0].plot((np.arange(len(out[1][0]))+1), out[1][0], label="train")
    ax[0].plot((np.arange(len(out[1][0]))+1), out[1][1], label="val")
    ax[1].plot((np.arange(len(out[1][0]))+1), out[1][0], label="train")
    ax[2].plot((np.arange(len(out[1][0]))+1), out[1][1], label="val")
    ax[3].plot((np.arange(len(out[1][0]))+1)[40:], out[1][0][40:], label="train")
    ax[4].plot((np.arange(len(out[1][0]))+1)[40:], out[1][1][40:], label="val")
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()
    ax[3].legend()
    ax[4].legend()
    plt.show()

In [ ]:
print(model)

In [ ]:
from utility_extended import *

In [ ]:
print(f"Train loss: {calcLossForDataset(model, dataloaders['train'], 1000, True)}\nVal loss: {calcLossForDataset(model, dataloaders['validation'], 1000, True)}\nTest loss: {calcLossForDataset(model, dataloaders['test'], 1000, True)}")

In [ ]:
torch.save(model, "./models/FlowFA-LowRank")

In [ ]:
from neuralpredictors.training import LongCycler
from torch.distributions import LowRankMultivariateNormal
def calcLossForDataset(model, dataset, neurons, samples_amount=100, in_bits=False):
    model.eval()
    with torch.no_grad():
        losses = 0
        samples_count = 0
        for batch_idx, (data_key, batch) in enumerate(LongCycler(dataset)):
            # data from batch
            #print(batch_idx)
            targets = batch[1]
            images = batch[0]
            
            # latent for log_likelihood
            mu = model.forward(images)
            
            if "Ident" in model.__class__.__name__:
                dist = torch.distributions.multivariate_normal.MultivariateNormal(mu, torch.eye(1000).to(mu.device))
            else:
                C, psi_diag = model.C_and_psi_diag
                dist = LowRankMultivariateNormal(mu, C.T, psi_diag)
            
            # get zero and non zero responses
            idx = targets <= torch.tensor(0)
            n_idx = torch.logical_not(idx)
            
            
            # calculate log_likelihood
            samples = torch.FloatTensor(samples_amount, targets.shape[0], 1000).uniform_(-1, 0)
            samples[:,n_idx] = targets[n_idx].cpu()    
            
            transformed_targets, logdet = model.sample_transform(samples.to(device))
            
            log_likelihood = dist.log_prob(transformed_targets.detach()) + logdet.detach().sum(dim=2)
            loss = -torch.sum(torch.mean(log_likelihood, dim=0))
            
            #print(samples.shape)
            #print(transformed_targets.shape)
            #print(log_likelihood.shape)
            #print(logdet.shape)
            #print(dist.log_prob(transformed_targets.detach()).shape)
            
            losses += loss.item()
            samples_count += len(batch[0])
            
            del samples, transformed_targets, logdet, log_likelihood, targets, images

        return losses / samples_count / neurons if in_bits==False else losses / samples_count / neurons / np.log(2)

In [ ]:
calcLossForDataset(model, dataloaders["train"], 1000, 100, True)

In [ ]:
#2.48